# Applying Image processing technique for enhancement and applying the trained model to segment images, visualized through a simple GUI:

In [1]:
# Import the tkinter library for GUI
import tkinter as tk  

# Import filedialog from tkinter for file selection
from tkinter import filedialog  

# Import Image and ImageTk from PIL for image handling
from PIL import Image, ImageTk, ImageEnhance  

# More important imports
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import os
import numpy as np
import glob
import pydicom 
import random
from pathlib import Path
from sklearn.model_selection import train_test_split
import shutil
import os
import cv2
from tensorflow.keras.models import load_model

In [ ]:
# Function to perform histogram equalization on an image
def histogram_equalization(img):
    # Calculate histogram and bins
    m = int(np.max(img))
    hist, bins = np.histogram(img, bins=m+1, range=(0, m+1))
    
    # Step 1: Calculate Probability Density Function (PDF)
    hist = hist / np.sum(hist)  # Normalize the histogram to make it a probability distribution
    
    # Step 2: Calculate Cumulative Distribution Function (CDF)
    cdf = np.cumsum(hist)
    
    # Step 3: Create a lookup table for mapping pixel values
    s_k = (255 * cdf).astype('uint8')  # Ensure s_k is of type uint8 for indexing
    
    # Generate a new equalized image using the lookup table
    img_new = s_k[img]
    return img_new

# Function to handle image selection
def select_image():
    global img_path, input_img_label  # Access global variables
    img_path = filedialog.askopenfilename()  # Open file dialog to select an image

    # Open the selected image using PIL
    img = load_dicom(img_path)
    img = np.asarray(img)
    img = (img - img.min()) / (img.max() - img.min())#.astype(np.uint8)
    img = 255*img
    img = img.astype(np.uint8)
    img = Image.fromarray(img)
    
    # Convert the image to 'RGB' mode before resizing
    img = img.convert('L')
    
    # Resize image to fit in the label
    img = img.resize((500, 500), Image.ANTIALIAS)

    # Convert the image to Tkinter PhotoImage format
    img = ImageTk.PhotoImage(img)

    # Update the image label in the GUI
    input_img_label.config(image=img)
    input_img_label.image = img

# Function to enhance the selected image using histogram equalization
def enhance_image():
    global img_path, enhanced_img_label

    if img_path:
        img = load_dicom(img_path)

        
        # Apply histogram equalization
        enhanced_img = histogram_equalization(np.array(img))

        # Normalize the enhanced image to [0, 255]
        enhanced_img = (enhanced_img - enhanced_img.min()) / (enhanced_img.max() - enhanced_img.min())
        enhanced_img = 255*enhanced_img
        enhanced_img = enhanced_img.astype(np.uint8)
        enhanced_img = Image.fromarray(enhanced_img)
        enhanced_img = enhanced_img.resize((500, 500), Image.ANTIALIAS)
        
        # Convert to Tkinter PhotoImage
        print(np.asarray(enhanced_img).shape, np.asarray(enhanced_img).min(), np.asarray(enhanced_img).max())
        enhanced_img_tk = ImageTk.PhotoImage(enhanced_img)

        enhanced_img_label.config(image=enhanced_img_tk)
        enhanced_img_label.image = enhanced_img_tk

# Function to segment the selected image using a loaded model
def segment_image():
    global img_path, segmented_img_label  # Access global variables
    if img_path:
        img = load_dicom(img_path)
        img=cv2.resize(img,(256,256))
        img=np.expand_dims(img, axis=-1)
        img=np.expand_dims(img, axis=0)
        print(img.shape)
        segmented_img= np.asarray(new_model.predict(img))[0:,:].astype(np.uint8)
        segmented_img = (segmented_img - segmented_img.min()) / (segmented_img.max() - segmented_img.min())
        segmented_img = 255*segmented_img
        segmented_img = segmented_img.astype(np.uint8)
        segmented_img = segmented_img[0][:,:,0]
        plt.imshow(segmented_img)
        plt.show()
        print(segmented_img.shape, segmented_img.min(), segmented_img.max())
        segmented_img = Image.fromarray(segmented_img)
        plt.imshow(segmented_img)
        plt.show()
        segmented_img = segmented_img.resize((500, 500), Image.ANTIALIAS)
        segmented_img
        plt.imshow(segmented_img)
        plt.show()
        segmented_img = segmented_img.convert('L')
        print(np.asarray(segmented_img).shape, np.asarray(segmented_img).min(), np.asarray(segmented_img).max())
        
        segmented_img = ImageTk.PhotoImage(segmented_img)  # Convert to Tkinter PhotoImage format
        print(np.asarray(segmented_img))
        segmented_img_label.config(image=segmented_img)  # Update the segmented image label
        segmented_img_label.image = segmented_img

In [6]:
# Load a pre-trained model from a specified file path
new_model = load_model(r"C:\Users\CLICK ONCE\Downloads\Unet_segmenter.h5", custom_objects={'dice_coef_loss': dice_coef_loss, 'dice_coef': dice_coef})

In [ ]:
# Visualize model summary
new_model.summary()

In [8]:
# Load trained model weights
new_model.load_weights(r"C:\Users\CLICK ONCE\Downloads\Unet_segmenter_weights.h5")

In [5]:
# Dice coefficient function used for evaluating model performance
def dice_coef(y_true, y_pred, smooth=0.001):
    # Calculate intersection and union of the true and predicted masks
    intersection = K.sum(y_true * y_pred, axis=[1, 2, 3])
    union = K.sum(y_true, axis=[1, 2, 3]) + K.sum(y_pred, axis=[1, 2, 3])
    
    # Calculate Dice coefficient using the intersection and union
    dice = (2. * intersection + smooth) / (union + smooth)
    
    return dice  # Dice coefficient calculated for evaluation (maximize for better performance)

# Custom loss function based on the Dice coefficient
def dice_coef_loss(y_true, y_pred):
    # Define loss as 1 minus the Dice coefficient to be used as the loss function during training
    return 1 - dice_coef(y_true, y_pred)

# Constants used for model training
EPOCHS = 100
BATCH_SIZE = 16
learning_rate = 1e-3
smooth = 0.001

In [9]:
def load_dicom(file_path):
    # Add your logic to read and process DICOM images
    # Return a single PIL Image object
    # Example:
    dicom_data = read_dicom_file(file_path)
    img_array = dicom_data.pixel_array  # Assuming pixel_array is an attribute of the DICOM object
    
    #img = Image.fromarray(img_array)
    return img_array


In [24]:
# Folder paths
DCM_PATH = r"C:\Users\CLICK ONCE\Downloads\Dicom files"

# List all X_train files
X_test_path = os.listdir(DCM_PATH)

# Initialize lists for X_train and Y_train data
X_test = []

# Iterate through X_train files
for x_file in X_test_path:
    # Load X_train
    x_data = load_dicom(os.path.join(DCM_PATH, x_file))
    x_data_resized = cv2.resize(x_data, (256, 256))

    # Append resized data to respective lists
    X_test.append(x_data_resized)
            
X_test = np.array(X_test)
X_test=np.expand_dims(X_test, axis=-1)

In [ ]:
# Tkinter window setup
root = tk.Tk()  # Create the main Tkinter window
root.title("Breast Cancer Detection Using Mammogram Images")  # Set the window title

# Set background color to light pink
root.configure(bg='#FFE6E6')

# Global variables
dycom_path = None  # Variable to store the path of the selected image

# Title for the whole window
app_title_label = tk.Label(root, text="Breast Cancer Detection Using Mammogram Images", bg='#FFE6E6', font=('Arial', 20, 'bold'))
app_title_label.grid(row=0, column=0, columnspan=3, pady=(10, 5))  # Place the label in the window

# Title for the input image
input_title_label = tk.Label(root, text="Input Image", bg='#FFE6E6', font=('Arial', 14, 'bold'))
input_title_label.grid(row=1, column=0, padx=5, pady=(10, 5))  # Place the label in the window

# Create a black placeholder screen for the input image
black_img = Image.new('RGB', (500, 500), color='black')  # Create a black image
black_img = ImageTk.PhotoImage(black_img)  # Convert to Tkinter PhotoImage format
input_img_label = tk.Label(root, image=black_img)  # Create label for the image
input_img_label.grid(row=2, column=0, padx=5)  # Place the label in the window

# Title for the segmented image
segmented_title_label = tk.Label(root, text="Segmented Image", bg='#FFE6E6', font=('Arial', 14, 'bold'))
segmented_title_label.grid(row=1, column=2, padx=5, pady=(10, 5))  # Place the label in the window

# Create a black placeholder screen for the segmented image
segmented_black_img = Image.new('RGB', (500, 500), color='black')  # Create a black image
segmented_black_img = ImageTk.PhotoImage(segmented_black_img)  # Convert to Tkinter PhotoImage format
segmented_img_label = tk.Label(root, image=segmented_black_img)  # Create label for the image
segmented_img_label.grid(row=2, column=2, padx=5)  # Place the label in the window

# Title for the segmented image
segmented_title_label = tk.Label(root, text="Enhanced Image", bg='#FFE6E6', font=('Arial', 14, 'bold'))
segmented_title_label.grid(row=1, column=1, padx=5, pady=(10, 5))  # Place the label in the window

# Create a black placeholder screen for the segmented image
enhanced_black_img = Image.new('RGB', (500, 500), color='black')  # Create a black image
enhanced_black_img = ImageTk.PhotoImage(enhanced_black_img)  # Convert to Tkinter PhotoImage format
enhanced_img_label = tk.Label(root, image=enhanced_black_img)  # Create label for the image
enhanced_img_label.grid(row=2, column=1, padx=5)  # Place the label in the window

# Button colors
button_color = '#FF6690'  # Define button color
button_font = ('Arial', 16, 'bold')  # Define button font

# Button to select an image
select_button = tk.Button(root, text="Select Image", command=select_image, bg=button_color, fg='white', padx=22, pady=5, font=button_font)
select_button.grid(row=4, column=0, pady=10)  # Place the button in the window

# Button to enhance the image
enhance_button = tk.Button(root, text="Enhance Image", command=enhance_image, bg=button_color, fg='white', padx=10, pady=5, font=button_font)
enhance_button.grid(row=4, column=1, pady=5)  # Place the button in the window

# Button to segment the image
segment_button = tk.Button(root, text="Segment Image", command=segment_image, bg=button_color, fg='white', padx=10, pady=5, font=button_font)
segment_button.grid(row=4, column=2, pady=5)  # Place the button in the window

# Configure rows and columns to expand
root.grid_rowconfigure(2, weight=1)
root.grid_columnconfigure(0, weight=1)
root.grid_columnconfigure(1, weight=1)

root.mainloop()  # Start the GUI event loop